In [1]:
# ~~~ Imports ~~~
import subprocess
import os
import hydra
from flows.utils.general_helpers import read_yaml_file
from flows.backends.api_info import ApiInfo
from flows.utils.general_helpers import find_replace_in_dict
from flows.flow_launchers import FlowLauncher
from flows.backends.api_info import ApiInfo
from flows import flow_verse
from flows import logging

logging.set_verbosity_info()

/opt/miniconda3/envs/flows/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Defining Functions Which Reproduce The Steps Of The Quick Start Tutorial (no modifications required)

In [2]:
# ~~~ Convenience function that install the dependencies for the Flows in this notebook ~~~
# Normally, you would install the dependencies in your terminal, as necessary. We do it here for you convenience

DEPENDENCIES_DICT = {
    "ChatFlowModule": "aiflows/ChatFlowModule",
    "ReAct": "aiflows/ControllerExecutorFlowModule",
    "ChatInteractiveFlowModule": "aiflows/ChatInteractiveFlowModule",
    "ChatWithDemonstrationsFlowModule": "aiflows/ChatWithDemonstrationsFlowModule",
    "AutoGPTFlowModule": "aiflows/AutoGPTFlowModule",
    "VisionFlowModule": "aiflows/VisionFlowModule",
}

def download_all_flow_dependencies(flow):
    """Download all dependencies for a flow from FlowVerse and pip requirements. 
    
    Note that this is a convenience function for this tutorial and is not meant to work doesn't work for all Flows on FlowVerse.
    
    :param flow: The name of the Flow to download dependencies for
    :type flow: str
    """
    # ~~~ Load Flow dependecies from FlowVerse ~~~
    dependencies = [
        {"url": DEPENDENCIES_DICT[flow], "revision": "main"},
    ]
    flow_verse.sync_dependencies(dependencies)
    
    # path to pip requirements file
    path_to_requirements = os.path.join("flow_modules" ,DEPENDENCIES_DICT[flow], "pip_requirements.txt")

    # download dependencies
    subprocess.check_call(["pip", "install", "-r", path_to_requirements], stdout=subprocess.DEVNULL)

# ~~~ Load Flow dependecies from FlowVerse and pip requirments ~~~
for key in DEPENDENCIES_DICT.keys():
    download_all_flow_dependencies(key)

[2023-12-07 12:55:21,680][flows.flow_verse.loading:775][INFO] - [<interactive>] started to sync flow module dependencies to /Users/josifosk/Documents/PhD/flows_wrapper/flows/examples/quick_start/flow_modules...
[2023-12-07 12:55:21,682][flows.flow_verse.loading:563][INFO] - aiflows/ChatFlowModule:main will be fetched from remote


Fetching 9 files: 100%|██████████| 9/9 [00:00<00:00, 6342.19it/s]


[2023-12-07 12:55:22,335][flows.flow_verse.loading:825][INFO] - [<interactive>] finished syncing


[2023-12-07 12:55:23,593][flows.flow_verse.loading:775][INFO] - [<interactive>] started to sync flow module dependencies to /Users/josifosk/Documents/PhD/flows_wrapper/flows/examples/quick_start/flow_modules...
[2023-12-07 12:55:23,594][flows.flow_verse.loading:563][INFO] - aiflows/ControllerExecutorFlowModule:main will be fetched from remote


Fetching 14 files: 100%|██████████| 14/14 [00:00<00:00, 5267.34it/s]


[2023-12-07 12:55:24,131][flows.flow_verse.loading:825][INFO] - [<interactive>] finished syncing


[2023-12-07 12:55:25,338][flows.flow_verse.loading:775][INFO] - [<interactive>] started to sync flow module dependencies to /Users/josifosk/Documents/PhD/flows_wrapper/flows/examples/quick_start/flow_modules...
[2023-12-07 12:55:25,340][flows.flow_verse.loading:563][INFO] - aiflows/ChatInteractiveFlowModule:main will be fetched from remote


...

[2023-12-07 12:55:27,932][flows.flow_verse.loading:825][INFO] - [<interactive>] finished syncing


[2023-12-07 12:55:29,142][flows.flow_verse.loading:775][INFO] - [<interactive>] started to sync flow module dependencies to /Users/josifosk/Documents/PhD/flows_wrapper/flows/examples/quick_start/flow_modules...
[2023-12-07 12:55:29,144][flows.flow_verse.loading:563][INFO] - aiflows/AutoGPTFlowModule:main will be fetched from remote


Fetching 9 files: 100%|██████████| 9/9 [00:00<00:00, 13802.10it/s]


[2023-12-07 12:55:29,681][flows.flow_verse.loading:825][INFO] - [<interactive>] finished syncing


[2023-12-07 12:55:31,099][flows.flow_verse.loading:775][INFO] - [<interactive>] started to sync flow module dependencies to /Users/josifosk/Documents/PhD/flows_wrapper/flows/examples/quick_start/flow_modules...
[2023-12-07 12:55:31,100][flows.flow_verse.loading:563][INFO] - aiflows/VisionFlowModule:main will be fetched from remote


Fetching 9 files: 100%|██████████| 9/9 [00:00<00:00, 11178.19it/s]


[2023-12-07 12:55:31,639][flows.flow_verse.loading:825][INFO] - [<interactive>] finished syncing




In [3]:
def run_flow(flow, data, api_information):
    """Loads the Flow config, instantiates the Flow and runs inference.
    
    :param flow: The name of the flow to run
    :type flow: str
    :param data: The input data to the flow
    :type data: dict
    :param api_information: The api information for the flow
    :type api_information: ApiInfo
    """
    
    flow_url = DEPENDENCIES_DICT[flow]

    # ~~~ Load Flow config ~~~
    path_to_config = os.path.join( "flow_modules" ,flow_url, "demo.yaml")
    cfg = read_yaml_file(path_to_config)

    # set the API information
    cfg = find_replace_in_dict(cfg, key_to_find="api_infos", new_value= api_information , current_path="")
  
    # ~~~ Instantiate the Flow ~~~
    flow_with_interfaces = {
        "flow": hydra.utils.instantiate(cfg["flow"], _recursive_=False, _convert_="partial"),
        "input_interface": None,
        "output_interface": None
    }
    
    # ~~~ Run inference ~~~
    path_to_output_file = None
    # path_to_output_file = "output.jsonl"  # Uncomment this line to save the output to disk

    _, outputs = FlowLauncher.launch(
        flow_with_interfaces=flow_with_interfaces, data=data, path_to_output_file=path_to_output_file
    )

    # ~~~ Print the output ~~~
    flow_output_data = outputs[0]
    print(flow_output_data)
    

### Setting you API Information

Set your API information here

In [4]:

# ~~~ Set the API information ~~~
# OpenAI backend
api_information = [ApiInfo(backend_used="openai", api_key=os.getenv("OPENAI_API_KEY"))]

# Azure backend
# api_information = ApiInfo(backend_used = "azure",
#                           api_base = os.getenv("AZURE_API_BASE"),
#                           api_key = os.getenv("AZURE_OPENAI_KEY"),
#                           api_version =  os.getenv("AZURE_API_VERSION") )

### Pick a Flow, some data and run it!

Set the `Flow` and `data` values, then run the cell! Examples for each flow from the FlowVerse are provided in the cell bellow. Uncomment the flow and data you wish to run. Here is the list of flows available for you to run:

* [ChatFlowModule](https://huggingface.co/aiflows/ChatFlowModule)

* [ReAct](https://huggingface.co/aiflows/ControllerExecutorFlowModule)

* [AutoGPTFlowModule](https://huggingface.co/aiflows/AutoGPTFlowModule)

* [ChatInteractiveFlowModule](https://huggingface.co/aiflows/ChatInteractiveFlowModule)

* [ChatWithDemonstrationsFlowModule](https://huggingface.co/aiflows/ChatWithDemonstrationsFlowModule)

* [VisionFlowModule](https://huggingface.co/aiflows/VisionFlowModule)


In [5]:
# Example for a simple chatbot
# [ChatFlowModule](https://huggingface.co/aiflows/ChatFlowModule)

flow = "ChatFlowModule"  
data = {"id": 0, "question": "Who was the NBA champion in 2023?"} 
run_flow(flow, data, api_information)

[2023-12-07 12:55:32,929][flows.flow_launchers.flow_API_launcher:194][INFO] - Running inference for ID (sample 0): 0
[{'api_output': "I'm sorry, but as an AI language model, I don't have access to real-time information or the ability to predict future events. As of now, I cannot provide you with the answer to who the NBA champion was in 2023. I recommend checking reliable sports news sources or conducting an internet search for the most up-to-date information."}]


In [6]:
# Example for ReAct (the simplest instance of a ControllerExecutorFlowModule)
flow = "ReAct"
data = {"id": 0, "goal": "Who was the NBA champion in 2023?"}
run_flow(flow, data, api_information)

[2023-12-07 12:55:32,948][flows.flow_verse.loading:775][INFO] - [aiflows.ControllerExecutorFlowModule] started to sync flow module dependencies to /Users/josifosk/Documents/PhD/flows_wrapper/flows/examples/quick_start/flow_modules...
[2023-12-07 12:55:33,230][flows.flow_verse.loading:594][WARNING] - [aiflows.ControllerExecutorFlowModule] aiflows/ChatFlowModule:main already synced, it will be overwritten by new revision aiflows/ChatFlowModule:a749ad10ed39776ba6721c37d0dc22af49ca0f17, are you sure? (Y/N)


Fetching 9 files: 100%|██████████| 9/9 [00:00<00:00, 24197.91it/s]


[2023-12-07 12:55:48,983][flows.flow_verse.loading:825][INFO] - [aiflows.ControllerExecutorFlowModule] finished syncing


[2023-12-07 12:55:48,988][flows.base_flows.circular:94][DEBUG] - output_msg_payload_processor [ControllerExecutorFlow.detect_finish_or_continue] registered
[2023-12-07 12:55:49,048][flows.base_flows.abstract:158][DEBUG] - Flow config not found at /Users/josifosk/Documents/PhD/flows_wrapper/flows/flows/base_flows/CompositeFlow.yaml.
[2023-12-07 12:55:49,057][flows.base_flows.abstract:158][DEBUG] - Flow config not found at /Users/josifosk/Documents/PhD/flows_wrapper/flows/flows/base_flows/AtomicFlow.yaml.
[2023-12-07 12:55:49,075][flows.base_flows.abstract:70][DEBUG] - Flow ControllerFlow instantiated with the following parameters:


CONFIG
├── private_keys
│   └── []                                                                                                         
├── keys_to_ignore_for_hash_flow_config
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_flow_state
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_input_data
│   └── []                                                                                                         
├── clear_flow_namespace_on_run_end
│   └── True                                                                                                       
├── enable_cache
│   └── True                                                                                                       
├── name
│   └── ControllerFlow                                                                                             
├── description
│   └── Proposes the next action to take towards achieving the goal, and prepares the input for the executor.      
├── n_api_retries
│   └── 6                                                                                                          
├── wait_time_between_retries
│   └── 20                                                                                                         
├── system_name
│   └── system                                                                                                     
├── user_name
│   └── user                                                                                                       
├── assistant_name
│   └── assistant                                                                                                  
├── backend
│   └── {'_target_': 'flows.backends.llm_lite.LiteLLMBackend', 'api_infos': [{'backend_used': 'openai', 'api_versio
├── system_message_prompt_template
│   └── {'_target_': 'flows.prompt_template.JinjaPrompt', 'template': 'You are a smart AI assistant. \n\nYour decis
├── init_human_message_prompt_template
│   └── {'_target_': 'flows.prompt_template.JinjaPrompt', 'template': 'Here is the goal you need to achieve:\n{{goa
├── human_message_prompt_template
│   └── {'_target_': 'flows.prompt_template.JinjaPrompt', 'template': 'Here is the response to your last action:\n{
├── input_interface_initialized
│   └── ['observation']                                                                                            
├── query_message_prompt_template
│   └── {'_target_': 'flows.prompt_template.JinjaPrompt'}                                                          
├── previous_messages
│   └── {'first_k': None, 'last_k': None}                                                                          
├── output_interface
│   └── ['thought', 'reasoning', 'plan', 'criticism', 'speak', 'command', 'command_args']                          
├── input_interface_non_initialized
│   └── ['goal']                                                                                                   
├── finish
│   └── {'description': 'Signal that the objective has been satisfied, and returns the answer to the user.', 'input
└── commands
    └── {'wiki_search': {'description[2;38;2

[2023-12-07 12:55:49,098][flows.base_flows.abstract:158][DEBUG] - Flow config not found at /Users/josifosk/Documents/PhD/flows_wrapper/flows/flows/base_flows/CompositeFlow.yaml.
[2023-12-07 12:55:49,100][flows.base_flows.abstract:158][DEBUG] - Flow config not found at /Users/josifosk/Documents/PhD/flows_wrapper/flows/flows/base_flows/AtomicFlow.yaml.
[2023-12-07 12:55:49,105][flows.base_flows.abstract:70][DEBUG] - Flow WikiSearchAtomicFlow instantiated with the following parameters:


...

[2023-12-07 12:55:49,116][flows.base_flows.abstract:70][DEBUG] - Flow unknown_name instantiated with the following parameters:


CONFIG
├── private_keys
│   └── []                                                                                                         
├── keys_to_ignore_for_hash_flow_config
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_flow_state
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_input_data
│   └── []                                                                                                         
├── clear_flow_namespace_on_run_end
│   └── True                                                                                                       
├── enable_cache
│   └── False                                                                                                      
├── input_interface
│   └── ['branch', 'branch_input_data']                                                                            
├── output_interface
│   └── ['branch_output_data']                                                                                     
└── subflows_config
    └── {'wiki_search': {'_target_': 'aiflows.ControllerExecutorFlowModule.WikiSearchAtomicFlow.instantiate_from_de

[2023-12-07 12:55:49,125][flows.base_flows.abstract:70][DEBUG] - Flow CtrlEx instantiated with the following parameters:


CONFIG
├── private_keys
│   └── []                                                                                                         
├── keys_to_ignore_for_hash_flow_config
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_flow_state
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_input_data
│   └── []                                                                                                         
├── clear_flow_namespace_on_run_end
│   └── True                                                                                                       
├── enable_cache
│   └── False                                                                                                      
├── max_rounds
│   └── 30                                                                                                         
├── early_exit_key
│   └── EARLY_EXIT                                                                                                 
├── topology
│   └── [{'goal': 'Select the next action and prepare the input for the executor.', 'input_interface': {'_target_':
├── name
│   └── CtrlEx                                                                                                     
├── description
│   └── ControllerExecutor (i.e., MRKL, ReAct) interaction implementation with Flows that approaches the problem so
├── input_interface
│   └── ['goal']                                                                                                   
├── output_interface
│   └── ['answer', 'status']                                                                                       
└── subflows_config
    └── {'Controller': {'_target_': 'aiflows.ControllerExecutorFlowModule.ControllerAtomicFlow.instantiate_from_def

[2023-12-07 12:55:49,156][flows.base_flows.abstract:309][DEBUG] - 
 ~~~ ResetMessageFull (UpdateMessage_FullReset): `Controller` ~~~
{
    "message_id": "dec0a908-847d-4fa2-badb-37c7c837c2c4",
    "created_at": "2023-12-07 11:55:49.156423000",
    "message_type": "UpdateMessage_FullReset",
    "created_by": "Launcher",
    "data": {
        "keys_deleted_from_namespace": []
    },
    "updated_flow": "Controller"
}
[2023-12-07 12:55:49,158][flows.base_flows.abstract:309][DEBUG] - 
 ~~~ ResetMessageFull (UpdateMessage_FullReset): `wiki_search` ~~~
{
    "message_id": "531e0576-c521-4444-a88a-e423b10c7588",
    "created_at": "2023-12-07 11:55:49.158075000",
    "message_type": "UpdateMessage_FullReset",
    "created_by": "Launcher",
    "data": {
        "keys_deleted_from_namespace": []
    },
    "updated_flow": "wiki_search"
}
[2023-12-07 12:55:49,159][flows.base_flows.abstract:309][DEBUG] - 
 ~~~ ResetMessageFull (UpdateMessage_FullReset): `Executor` ~~~
{
    "message_id": "4bc6bdaa

In [7]:
# Example for AutoGPTFlowModule
flow = "AutoGPTFlowModule"
data = {"id": 0, "goal": "Who was the NBA champion in 2023?"}
run_flow(flow, data, api_information)

[2023-12-07 12:56:00,958][flows.flow_verse.loading:775][INFO] - [aiflows.AutoGPTFlowModule] started to sync flow module dependencies to /Users/josifosk/Documents/PhD/flows_wrapper/flows/examples/quick_start/flow_modules...
[2023-12-07 12:56:01,287][flows.flow_verse.loading:594][WARNING] - [aiflows.AutoGPTFlowModule] aiflows/ControllerExecutorFlowModule:main already synced, it will be overwritten by new revision aiflows/ControllerExecutorFlowModule:09cda9615e5c48ae18e2c1244519ed7321145187, are you sure? (Y/N)


Fetching 14 files: 100%|██████████| 14/14 [00:00<00:00, 16338.41it/s]


[2023-12-07 12:57:56,392][flows.flow_verse.loading:563][INFO] - aiflows/HumanStandardInputFlowModule:5683a922372c5fa90be9f6447d6662d8d80341fc will be fetched from remote


Fetching 8 files: 100%|██████████| 8/8 [00:00<00:00, 20447.55it/s]


[2023-12-07 12:57:56,962][flows.flow_verse.loading:563][INFO] - aiflows/VectorStoreFlowModule:692f5d1b55936d813d4f41e8b0ec11754c7da9ac will be fetched from remote


...

[2023-12-07 12:58:01,004][flows.base_flows.abstract:70][DEBUG] - Flow AutoGPTFlow instantiated with the following parameters:


CONFIG
├── private_keys
│   └── []                                                                                                         
├── keys_to_ignore_for_hash_flow_config
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_flow_state
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_input_data
│   └── []                                                                                                         
├── clear_flow_namespace_on_run_end
│   └── True                                                                                                       
├── enable_cache
│   └── False                                                                                                      
├── max_rounds
│   └── 30                                                                                                         
├── early_exit_key
│   └── EARLY_EXIT                                                                                                 
├── topology
│   └── [{'goal': 'Retrieve relevant information from memory.', 'input_interface': {'_target_': 'AutoGPTFlow.prepar
├── name
│   └── AutoGPTFlow                                                                                                
├── description
│   └── An example implementation of AutoGPT with Flows.                                                           
├── input_interface
│   └── ['goal']                                                                                                   
├── output_interface
│   └── ['answer', 'status']                                                                                       
└── subflows_config
    └── {'Controller': {'_target_': 'aiflows.ControllerExecutorFlowModule.ControllerAtomicFlow.instantiate_from_def

[2023-12-07 12:58:01,017][flows.base_flows.abstract:309][DEBUG] - 
 ~~~ ResetMessageFull (UpdateMessage_FullReset): `Controller` ~~~
{
    "message_id": "2ae4773a-0430-4513-be14-b9f4c30e5499",
    "created_at": "2023-12-07 11:58:01.017637000",
    "message_type": "UpdateMessage_FullReset",
    "created_by": "Launcher",
    "data": {
        "keys_deleted_from_namespace": []
    },
    "updated_flow": "Controller"
}
[2023-12-07 12:58:01,018][flows.base_flows.abstract:309][DEBUG] - 
 ~~~ ResetMessageFull (UpdateMessage_FullReset): `wiki_search` ~~~
{
    "message_id": "8e603e9a-c996-404e-aca6-8e5342fa0956",
    "created_at": "2023-12-07 11:58:01.018415000",
    "message_type": "UpdateMessage_FullReset",
    "created_by": "Launcher",
    "data": {
        "keys_deleted_from_namespace": []
    },
    "updated_flow": "wiki_search"
}
[2023-12-07 12:58:01,018][flows.base_flows.abstract:309][DEBUG] - 
 ~~~ ResetMessageFull (UpdateMessage_FullReset): `Executor` ~~~
{
    "message_id": "e9c4d823

Number of requested results 2 is greater than number of elements in index 1, updating n_results = 1


[2023-12-07 12:58:33,560][flows.base_flows.abstract:309][DEBUG] - 
 ~~~ ResetMessageNamespaceOnly (UpdateMessage_NamespaceReset): `Memory` ~~~
{
    "message_id": "e4e4540e-6560-4887-aa1f-c3c56b484060",
    "created_at": "2023-12-07 11:58:33.560166000",
    "message_type": "UpdateMessage_NamespaceReset",
    "created_by": "Memory",
    "data": {
        "keys_deleted_from_namespace": []
    },
    "updated_flow": "Memory"
}
[2023-12-07 12:58:33,560][flows.base_flows.abstract:309][DEBUG] - 
 ~~~ OutputMessage: `Memory` --> `AutoGPTFlow` ~~~
{
    "message_id": "87a56e1b-24b4-41d2-913f-0efe92433ec3",
    "created_at": "2023-12-07 11:58:33.560071000",
    "message_type": "OutputMessage",
    "created_by": "Memory",
    "data": {
        "output_data": {
            "retrieved": [
                [
                    "\n            == Goal == \n            Who was the NBA champion in 2023?\n            \n            == Command ==\n            wiki_search\n            == Args\n            

In [8]:
# Example for ChatInteractiveFlowModule
flow = "ChatInteractiveFlowModule"
data = {"id": 0} # you will be able to ask questions once it is running
run_flow(flow, data, api_information)

[2023-12-07 12:59:20,865][flows.flow_verse.loading:775][INFO] - [aiflows.ChatInteractiveFlowModule] started to sync flow module dependencies to /Users/josifosk/Documents/PhD/flows_wrapper/flows/examples/quick_start/flow_modules...
[2023-12-07 12:59:21,171][flows.flow_verse.loading:608][INFO] - aiflows/ChatFlowModule:a749ad10ed39776ba6721c37d0dc22af49ca0f17 already synced, skip
[2023-12-07 12:59:21,477][flows.flow_verse.loading:608][INFO] - aiflows/HumanStandardInputFlowModule:5683a922372c5fa90be9f6447d6662d8d80341fc already synced, skip
[2023-12-07 12:59:21,478][flows.flow_verse.loading:825][INFO] - [aiflows.ChatInteractiveFlowModule] finished syncing


[2023-12-07 12:59:21,480][flows.base_flows.abstract:158][DEBUG] - Flow config not found at /Users/josifosk/Documents/PhD/flows_wrapper/flows/flows/base_flows/CompositeFlow.yaml.
[2023-12-07 12:59:21,489][flows.base_flows.abstract:158][DEBUG] - Flow config not found at /Users/josifosk/Documents/PhD/flows_wrapper/flows/flows/base_flows/At

CONFIG
├── private_keys
│   └── []                                                                                                         
├── keys_to_ignore_for_hash_flow_config
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_flow_state
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_input_data
│   └── []                                                                                                         
├── clear_flow_namespace_on_run_end
│   └── True                                                                                                       
├── enable_cache
│   └── True                                                                                                       
├── name
│   └── ChatAtomicFlow                                                                                             
├── description
│   └── Flow which uses as tool an LLM though an API                                                               
├── n_api_retries
│   └── 6                                                                                                          
├── wait_time_between_retries
│   └── 20                                                                                                         
├── system_name
│   └── system                                                                                                     
├── user_name
│   └── user                                                                                                       
├── assistant_name
│   └── assistant                                                                                                  
├── backend
│   └── {'_target_': 'flows.backends.llm_lite.LiteLLMBackend', 'api_infos': [{'backend_used': 'openai', 'api_versio
├── system_message_prompt_template
│   └── {'_target_': 'flows.prompt_template.JinjaPrompt'}                                                          
├── init_human_message_prompt_template
│   └── {'_target_': 'flows.prompt_template.JinjaPrompt'}                                                          
├── human_message_prompt_template
│   └── {'_target_': 'flows.prompt_template.JinjaPrompt', 'template': '{{query}}', 'input_variables': ['query']}   
├── input_interface_initialized
│   └── ['query']                                                                                                  
├── query_message_prompt_template
│   └── {'_target_': 'flows.prompt_template.JinjaPrompt'}                                                          
├── previous_messages
│   └── {'first_k': None, 'last_k': None}                                                                          
├── output_interface
│   └── ['api_output']                                                                                             
└── input_interface_non_initialized
    └── []                                                                                                         

[2023-12-07 12:59:21,510][flows.base_flows.abstract:158][DEBUG] - Flow config not found at /Users/josifosk/Documents/PhD/flows_wrapper/flows/flows/base_flows/AtomicFlow.yaml.
[2023-12-07 12:59:21,513][flows.base_flows.abstract:70][DEBUG] - Flow unknown_name instantiated with the following parameters:


CONFIG
├── private_keys
│   └── []                                                                                                         
├── keys_to_ignore_for_hash_flow_config
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_flow_state
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_input_data
│   └── []                                                                                                         
├── clear_flow_namespace_on_run_end
│   └── True                                                                                                       
├── enable_cache
│   └── False                                                                                                      
├── end_of_input_string
│   └── EOI                                                                                                        
├── description
│   └── Reads input from the user's standard input.                                                                
├── query_message_prompt_template
│   └── {'_target_': 'flows.prompt_template.JinjaPrompt', 'template': '{{api_output}}\n\nTo end an Interaction, typ
├── request_multi_line_input_flag
│   └── False                                                                                                      
├── input_interface
│   └── ['api_output']                                                                                             
└── output_interface
    └── ['human_input']                                                                                            

[2023-12-07 12:59:21,519][flows.base_flows.abstract:70][DEBUG] - Flow ChatInteractiveFlow instantiated with the following parameters:


CONFIG
├── private_keys
│   └── []                                                                                                         
├── keys_to_ignore_for_hash_flow_config
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_flow_state
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_input_data
│   └── []                                                                                                         
├── clear_flow_namespace_on_run_end
│   └── True                                                                                                       
├── enable_cache
│   └── False                                                                                                      
├── max_rounds
│   └── 5                                                                                                          
├── early_exit_key
│   └── end_of_interaction                                                                                         
├── topology
│   └── [{'goal': 'Query the assistant', 'input_interface': {'_target_': 'flows.interfaces.KeyInterface', 'addition
├── name
│   └── ChatInteractiveFlow                                                                                        
├── description
│   └── Flow that enables chatting between a ChatAtomicFlow and a user providing the input.                        
├── input_interface
│   └── {'_target_': 'flows.interfaces.KeyInterface'}                                                              
├── output_interface
│   └── ['end_of_interaction', 'answer']                                                                           
└── subflows_config
    └── {'Assistant': {'_target_': 'aiflows.ChatFlowModule.ChatAtomicFlow.instantiate_from_default_config', 'backen

[2023-12-07 12:59:21,534][flows.base_flows.abstract:309][DEBUG] - 
 ~~~ ResetMessageFull (UpdateMessage_FullReset): `Assistant` ~~~
{
    "message_id": "146c447d-3ea5-44a7-9396-59b7d6869a7e",
    "created_at": "2023-12-07 11:59:21.534511000",
    "message_type": "UpdateMessage_FullReset",
    "created_by": "Launcher",
    "data": {
        "keys_deleted_from_namespace": []
    },
    "updated_flow": "Assistant"
}
[2023-12-07 12:59:21,535][flows.base_flows.abstract:309][DEBUG] - 
 ~~~ ResetMessageFull (UpdateMessage_FullReset): `User` ~~~
{
    "message_id": "87bd6da1-24ee-4329-b4e6-ed8d318704f0",
    "created_at": "2023-12-07 11:59:21.535167000",
    "message_type": "UpdateMessage_FullReset",
    "created_by": "Launcher",
    "data": {
        "keys_deleted_from_namespace": []
    },
    "updated_flow": "User"
}
[2023-12-07 12:59:21,535][flows.base_flows.abstract:309][DEBUG] - 
 ~~~ ResetMessageFull (UpdateMessage_FullReset): `ChatInteractiveFlow` ~~~
{
    "message_id": "66caae1e-6af0

In [9]:
# Example for ChatWithDemonstrationsFlowModule
flow = "ChatWithDemonstrationsFlowModule"
data = {"id": 0, "question": "What's the capital of France?"}  # This can be a list of samples
run_flow(flow, data, api_information)

[2023-12-07 13:00:06,822][flows.flow_verse.loading:775][INFO] - [aiflows.ChatWithDemonstrationsFlowModule] started to sync flow module dependencies to /Users/josifosk/Documents/PhD/flows_wrapper/flows/examples/quick_start/flow_modules...
[2023-12-07 13:00:07,079][flows.flow_verse.loading:608][INFO] - aiflows/ChatFlowModule:a749ad10ed39776ba6721c37d0dc22af49ca0f17 already synced, skip
[2023-12-07 13:00:07,081][flows.flow_verse.loading:825][INFO] - [aiflows.ChatWithDemonstrationsFlowModule] finished syncing


[2023-12-07 13:00:07,084][flows.base_flows.abstract:158][DEBUG] - Flow config not found at /Users/josifosk/Documents/PhD/flows_wrapper/flows/flows/base_flows/CompositeFlow.yaml.
[2023-12-07 13:00:07,091][flows.base_flows.abstract:158][DEBUG] - Flow config not found at /Users/josifosk/Documents/PhD/flows_wrapper/flows/flows/base_flows/AtomicFlow.yaml.
[2023-12-07 13:00:07,098][flows.base_flows.abstract:70][DEBUG] - Flow DemonstrationAtomicFlow instantiated with the following paramete

CONFIG
├── private_keys
│   └── []                                                                                                         
├── keys_to_ignore_for_hash_flow_config
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_flow_state
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_input_data
│   └── []                                                                                                         
├── clear_flow_namespace_on_run_end
│   └── True                                                                                                       
├── enable_cache
│   └── False                                                                                                      
├── name
│   └── DemonstrationAtomicFlow                                                                                    
├── description
│   └── A flow which returns Demonstrations                                                                        
├── data
│   └── [{'query_data': {'query': 'What is the capital of Turkey?'}, 'response_data': {'response': 'Istambul, my si
├── params
│   └── {'data_dir': None, 'demonstrations_id': 'my_sir_demo', 'ids_to_keep': None, 'demonstrations_k': None}      
├── query_prompt_template
│   └── {'_target_': 'flows.prompt_template.JinjaPrompt', 'template': 'Answer the following question: {{query}}', '
└── response_prompt_template
    └── {'_target_': 'flows.prompt_template.JinjaPrompt', 'template': '{{response}}', 'input_variables': ['response

[2023-12-07 13:00:07,107][flows.base_flows.abstract:158][DEBUG] - Flow config not found at /Users/josifosk/Documents/PhD/flows_wrapper/flows/flows/base_flows/AtomicFlow.yaml.
[2023-12-07 13:00:07,119][flows.base_flows.abstract:70][DEBUG] - Flow SimpleQA_Flow instantiated with the following parameters:


CONFIG
├── private_keys
│   └── []                                                                                                         
├── keys_to_ignore_for_hash_flow_config
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_flow_state
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_input_data
│   └── []                                                                                                         
├── clear_flow_namespace_on_run_end
│   └── True                                                                                                       
├── enable_cache
│   └── True                                                                                                       
├── name
│   └── SimpleQA_Flow                                                                                              
├── description
│   └── Flow which uses as tool an LLM though an API                                                               
├── n_api_retries
│   └── 6                                                                                                          
├── wait_time_between_retries
│   └── 20                                                                                                         
├── system_name
│   └── system                                                                                                     
├── user_name
│   └── user                                                                                                       
├── assistant_name
│   └── assistant                                                                                                  
├── backend
│   └── {'_target_': 'flows.backends.llm_lite.LiteLLMBackend', 'api_infos': [{'backend_used': 'openai', 'api_versio
├── system_message_prompt_template
│   └── {'_target_': 'flows.prompt_template.JinjaPrompt', 'template': 'You are a helpful chatbot that truthfully an
├── init_human_message_prompt_template
│   └── {'_target_': 'flows.prompt_template.JinjaPrompt', 'template': 'Answer the following question: {{question}}'
├── human_message_prompt_template
│   └── {'_target_': 'flows.prompt_template.JinjaPrompt', 'template': '{{query}}', 'input_variables': ['query']}   
├── input_interface_initialized
│   └── ['query']                                                                                                  
├── query_message_prompt_template
│   └── {'_target_': 'flows.prompt_template.JinjaPrompt'}                                                          
├── previous_messages
│   └── {'first_k': None, 'last_k': None}                                                                          
├── output_interface
│   └── ['api_output']                                                                                             
└── input_interface_non_initialized
    └── ['question']                                                                                               

[2023-12-07 13:00:07,131][flows.base_flows.abstract:70][DEBUG] - Flow SimpleQA_Flow_with_Demonstrations instantiated with the following parameters:


CONFIG
├── private_keys
│   └── []                                                                                                         
├── keys_to_ignore_for_hash_flow_config
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_flow_state
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_input_data
│   └── []                                                                                                         
├── clear_flow_namespace_on_run_end
│   └── True                                                                                                       
├── enable_cache
│   └── False                                                                                                      
├── max_rounds
│   └── 1                                                                                                          
├── early_exit_key
│   └── EARLY_EXIT                                                                                                 
├── topology
│   └── [{'goal': 'Get Demonstrations', 'input_interface': {'_target_': 'flows.interfaces.KeyInterface'}, 'flow': '
├── name
│   └── SimpleQA_Flow_with_Demonstrations                                                                          
├── description
│   └── A sequential flow that answers questions with demonstrations                                               
├── subflows_config
│   └── {'demonstration_flow': {'_target_': 'aiflows.ChatWithDemonstrationsFlowModule.DemonstrationsAtomicFlow.inst
├── input_interface_initialized
│   └── ['query']                                                                                                  
├── output_interface
│   └── ['answer']                                                                                                 
└── input_interface
    └── ['questions']                                                                                              

[2023-12-07 13:00:07,142][flows.base_flows.abstract:309][DEBUG] - 
 ~~~ ResetMessageFull (UpdateMessage_FullReset): `demonstration_flow` ~~~
{
    "message_id": "72bda6c2-1224-4f3d-917b-e62e086c422a",
    "created_at": "2023-12-07 12:00:07.141981000",
    "message_type": "UpdateMessage_FullReset",
    "created_by": "Launcher",
    "data": {
        "keys_deleted_from_namespace": []
    },
    "updated_flow": "demonstration_flow"
}
[2023-12-07 13:00:07,143][flows.base_flows.abstract:309][DEBUG] - 
 ~~~ ResetMessageFull (UpdateMessage_FullReset): `chat_flow` ~~~
{
    "message_id": "da312ba5-ba86-46e3-9fcc-ba0fe531a9b7",
    "created_at": "2023-12-07 12:00:07.143770000",
    "message_type": "UpdateMessage_FullReset",
    "created_by": "Launcher",
    "data": {
        "keys_deleted_from_namespace": []
    },
    "updated_flow": "chat_flow"
}
[2023-12-07 13:00:07,146][flows.base_flows.abstract:309][DEBUG] - 
 ~~~ ResetMessageFull (UpdateMessage_FullReset): `SimpleQA_Flow_with_Demonstratio

In [10]:
# Example for ChatInteractiveFlowModule
flow = "ChatInteractive/FlowModule"
data = {"id": 0} # you will be able to ask questions once it is running
run_flow(flow, data, api_information)

KeyError: 'ChatInteractive/FlowModule'

In [ ]:
# Example for ChatWithDemonstrationsFlowModule
flow = "ChatWithDemonstrationsFlowModule"
data = {"id": 0, "question": "What's the capital of France?"}  # This can be a list of samples
run_flow(flow, data, api_information)

[2023-12-07 12:53:21,081][flows.base_flows.abstract:158][DEBUG] - Flow config not found at /Users/josifosk/Documents/PhD/flows_wrapper/flows/flows/base_flows/CompositeFlow.yaml.
[2023-12-07 12:53:21,089][flows.base_flows.abstract:158][DEBUG] - Flow config not found at /Users/josifosk/Documents/PhD/flows_wrapper/flows/flows/base_flows/AtomicFlow.yaml.
[2023-12-07 12:53:21,096][flows.base_flows.abstract:70][DEBUG] - Flow DemonstrationAtomicFlow instantiated with the following parameters:


CONFIG
├── private_keys
│   └── []                                                                                                         
├── keys_to_ignore_for_hash_flow_config
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_flow_state
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_input_data
│   └── []                                                                                                         
├── clear_flow_namespace_on_run_end
│   └── True                                                                                                       
├── enable_cache
│   └── False                                                                                                      
├── name
│   └── DemonstrationAtomicFlow                                                                                    
├── description
│   └── A flow which returns Demonstrations                                                                        
├── data
│   └── [{'query_data': {'query': 'What is the capital of Turkey?'}, 'response_data': {'response': 'Istambul, my si
├── params
│   └── {'data_dir': None, 'demonstrations_id': 'my_sir_demo', 'ids_to_keep': None, 'demonstrations_k': None}      
├── query_prompt_template
│   └── {'_target_': 'flows.prompt_template.JinjaPrompt', 'template': 'Answer the following question: {{query}}', '
└── response_prompt_template
    └── {'_target_': 'flows.prompt_template.JinjaPrompt', 'template': '{{response}}', 'input_variables': ['response

[2023-12-07 12:53:21,106][flows.base_flows.abstract:158][DEBUG] - Flow config not found at /Users/josifosk/Documents/PhD/flows_wrapper/flows/flows/base_flows/AtomicFlow.yaml.
[2023-12-07 12:53:21,118][flows.base_flows.abstract:70][DEBUG] - Flow SimpleQA_Flow instantiated with the following parameters:


CONFIG
├── private_keys
│   └── []                                                                                                         
├── keys_to_ignore_for_hash_flow_config
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_flow_state
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_input_data
│   └── []                                                                                                         
├── clear_flow_namespace_on_run_end
│   └── True                                                                                                       
├── enable_cache
│   └── True                                                                                                       
├── name
│   └── SimpleQA_Flow                                                                                              
├── description
│   └── Flow which uses as tool an LLM though an API                                                               
├── n_api_retries
│   └── 6                                                                                                          
├── wait_time_between_retries
│   └── 20                                                                                                         
├── system_name
│   └── system                                                                                                     
├── user_name
│   └── user                                                                                                       
├── assistant_name
│   └── assistant                                                                                                  
├── backend
│   └── {'_target_': 'flows.backends.llm_lite.LiteLLMBackend', 'api_infos': [{'backend_used': 'openai', 'api_versio
├── system_message_prompt_template
│   └── {'_target_': 'flows.prompt_template.JinjaPrompt', 'template': 'You are a helpful chatbot that truthfully an
├── init_human_message_prompt_template
│   └── {'_target_': 'flows.prompt_template.JinjaPrompt', 'template': 'Answer the following question: {{question}}'
├── human_message_prompt_template
│   └── {'_target_': 'flows.prompt_template.JinjaPrompt', 'template': '{{query}}', 'input_variables': ['query']}   
├── input_interface_initialized
│   └── ['query']                                                                                                  
├── query_message_prompt_template
│   └── {'_target_': 'flows.prompt_template.JinjaPrompt'}                                                          
├── previous_messages
│   └── {'first_k': None, 'last_k': None}                                                                          
├── output_interface
│   └── ['api_output']                                                                                             
└── input_interface_non_initialized
    └── ['question']                                                                                               

[2023-12-07 12:53:21,128][flows.base_flows.abstract:70][DEBUG] - Flow SimpleQA_Flow_with_Demonstrations instantiated with the following parameters:


CONFIG
├── private_keys
│   └── []                                                                                                         
├── keys_to_ignore_for_hash_flow_config
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_flow_state
│   └── ['name', 'description', 'api_keys', 'api_information', 'private_keys']                                     
├── keys_to_ignore_for_hash_input_data
│   └── []                                                                                                         
├── clear_flow_namespace_on_run_end
│   └── True                                                                                                       
├── enable_cache
│   └── False                                                                                                      
├── max_rounds
│   └── 1                                                                                                          
├── early_exit_key
│   └── EARLY_EXIT                                                                                                 
├── topology
│   └── [{'goal': 'Get Demonstrations', 'input_interface': {'_target_': 'flows.interfaces.KeyInterface'}, 'flow': '
├── name
│   └── SimpleQA_Flow_with_Demonstrations                                                                          
├── description
│   └── A sequential flow that answers questions with demonstrations                                               
├── subflows_config
│   └── {'demonstration_flow': {'_target_': 'aiflows.ChatWithDemonstrationsFlowModule.DemonstrationsAtomicFlow.inst
├── input_interface_initialized
│   └── ['query']                                                                                                  
├── output_interface
│   └── ['answer']                                                                                                 
└── input_interface
    └── ['questions']                                                                                              

[2023-12-07 12:53:21,142][flows.base_flows.abstract:309][DEBUG] - 
 ~~~ ResetMessageFull (UpdateMessage_FullReset): `demonstration_flow` ~~~
{
    "message_id": "0b1031bf-920b-4cbc-a5c9-47ba2174fead",
    "created_at": "2023-12-07 11:53:21.142140000",
    "message_type": "UpdateMessage_FullReset",
    "created_by": "Launcher",
    "data": {
        "keys_deleted_from_namespace": []
    },
    "updated_flow": "demonstration_flow"
}
[2023-12-07 12:53:21,143][flows.base_flows.abstract:309][DEBUG] - 
 ~~~ ResetMessageFull (UpdateMessage_FullReset): `chat_flow` ~~~
{
    "message_id": "9244482a-d84e-4b4f-9b53-d4d0b7ca4841",
    "created_at": "2023-12-07 11:53:21.143317000",
    "message_type": "UpdateMessage_FullReset",
    "created_by": "Launcher",
    "data": {
        "keys_deleted_from_namespace": []
    },
    "updated_flow": "chat_flow"
}
[2023-12-07 12:53:21,144][flows.base_flows.abstract:309][DEBUG] - 
 ~~~ ResetMessageFull (UpdateMessage_FullReset): `SimpleQA_Flow_with_Demonstratio

In [ ]:
# Example for VisionFlowModule
flow = "VisionFlowModule"
url_image = {"type": "url",
            "image": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"}
data = {"id": 0, "query": "What’s in this image?", "data": {"images": [url_image]}}  # This can be a list of samples
run_flow(flow, data, api_information)